In [1]:
rm(list=ls())

suppressMessages(library(MERINGUE))
library(ComplexHeatmap)

Loading required package: grid

ComplexHeatmap version 2.11.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite:
Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
  genomic data. Bioinformatics 2016.

The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))




In [2]:
setwd('~/Spatial_project')
all_bc_meta = read.csv('~/Spatial_project/data/all_bc_meta_new.csv')
slisa_df_all = data.frame()
source_all = c()
bn_df_all = data.frame()
ct_all = c()
slisa_all = data.frame()

sample_ids = {c('MBC_1A',
             'MBC_1B',
             #'MBC_2A',
             #'MBC_2B',
             #'MBC_3A',
             #'MBC_3B',
             #'MBC_4A',
             'MBC_4B',
             #'CID4465',
             'CID444971',
             #'1142243F',
             '1160920F')
}


In [3]:
for (sample_id in sample_ids){
  print(paste0('working on',sample_id))
  bn_df = read.csv(paste0('~/Spatial_project/results/02_PoE_factor/',sample_id,'nb_file.csv'),row.names = 1)
  pos = read.csv(paste0('~/Spatial_project/data/',sample_id,'/spatial/tissue_positions_list.csv'),row.names = 1,header = FALSE)
  pos = pos[row.names(bn_df),]
  # normalize data
  bn_df_n = bn_df
  for(col in colnames(bn_df)){
    bn_df_n[,col] = bn_df[,col]/max(max(bn_df))
  }
  row.names(bn_df_n)= paste0(row.names(bn_df),'_',sample_id)
  row.names(pos)= paste0(row.names(pos),'_',sample_id)
  bn_df_all = rbind(bn_df_all,bn_df_n)
  
  source_all = c(source_all,rep(all_bc_meta[which(all_bc_meta[,'sample_id']==sample_id),'sample_id'],nrow(bn_df)))
  ct_all = c(ct_all,rep(all_bc_meta[which(all_bc_meta[,'sample_id']==sample_id),'cancer_type'],nrow(bn_df))) 
  weight <- getSpatialNeighbors(pos[row.names(bn_df_n),c('V5','V6')])
  tmp = data.frame(row.names = row.names(bn_df_n))
  
  for(bn in colnames(bn_df)){
    gexp <- bn_df_n[,bn]#/max(bn_df_2A_n[,bn])#bn_all_n[,col] = bn_all[,col]/max(bn_all[,col])
    names(gexp) = row.names(bn_df_n)
    slisa <- signedLisa(gexp, weight)
    tmp[,bn] = slisa
  }
  slisa_all = rbind(slisa_all,tmp)
}

[1] "working onMBC_1A"
[1] "working onMBC_1B"
[1] "working onMBC_4B"
[1] "working onCID444971"
[1] "working on1160920F"


In [4]:
write.csv(bn_df_all,file=paste0('~/Spatial_project/results/03_hubs/','all_bn_tumors.csv'))
write.csv(source_all,file=paste0('~/Spatial_project/results/03_hubs/','all_source_tumors.csv'))
write.csv(ct_all,file=paste0('~/Spatial_project/results/03_hubs/','all_ct_tumors.csv'))
write.csv(slisa_all,file=paste0('~/Spatial_project/results/03_hubs/','all_slisa_tumors.csv'))